<a href="https://colab.research.google.com/github/dnrocha/deepts/blob/master/C%C3%B3digo_Deep_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> INSTITUTO FEDERAL DO NORTE DE MINAS GERAIS - CAMPUS MONTES CLAROS
>
>
> **Código de Aprendizagem Profunda Para Transferência de Estilo entre Imagens Digitais**
>
>
>
> *Autor: Daniel Antunes Rocha*
